In [5]:
import pandas as pd

In [6]:
foncier2014 = pd.read_csv('../../data/foncier/full_2014.csv',sep=',',dtype= {'code_commune' : str,'code_postal' : str})

In [11]:
foncier2014.head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2014-1,2014-01-09,1,Vente,251500.0,532.0,NaN,RTE DE MEZERIAT,0006,01310,...,Maison,147.0,5.0,S,sols,NaN,NaN,1501.0,5.119191,46.247576
1,2014-2,2014-01-09,1,Vente,174500.0,22.0,NaN,BD MAL LECLERC,2540,01000,...,Appartement,80.0,2.0,NaN,NaN,NaN,NaN,NaN,5.218208,46.204017
2,2014-2,2014-01-09,1,Vente,174500.0,22.0,NaN,BD MAL LECLERC,2540,01000,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,5.218208,46.204017
3,2014-3,2014-01-02,1,Vente,157500.0,1389.0,NaN,RTE DE STRASBOURG,0201,01440,...,NaN,NaN,NaN,S,sols,NaN,NaN,24.0,5.262505,46.236179
4,2014-3,2014-01-02,1,Vente,157500.0,1369.0,NaN,RTE DE STRASBOURG,0201,01440,...,Maison,103.0,4.0,S,sols,NaN,NaN,1569.0,5.262756,46.236276


In [12]:
foncier2014['valeur_fonciere'].isna().sum()

28279

In [13]:
res = foncier2014['type_local'].unique()
res

array(['Maison', 'Appartement', 'Dépendance', nan,
       'Local industriel. commercial ou assimilé'], dtype=object)

In [14]:
foncier2014.type_local.value_counts(dropna=False)

NaN                                         1169548
Maison                                       470502
Appartement                                  441343
Dépendance                                   344057
Local industriel. commercial ou assimilé      91238
Name: type_local, dtype: int64

In [15]:
immo2014 = foncier2014

# pour faire un premier tri dans les colonnes

In [16]:
cols = ['id_mutation', 'date_mutation', 'numero_disposition',
       'nature_mutation', 'valeur_fonciere', 
       'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'id_parcelle',
        'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture',
    'surface_terrain', 'longitude', 'latitude']

In [17]:
immo2014 = immo2014[cols]

In [18]:
immo2014[immo2014.nombre_pieces_principales >= 90]

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,code_postal,code_commune,nom_commune,code_departement,id_parcelle,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,surface_terrain,longitude,latitude
240460,2014-95152,2014-02-28,1,Vente,49263.00,13001,13201,Marseille 1er Arrondissement,13,132018030A0235,2.0,Appartement,1.0,91.0,NaN,NaN,NaN,5.379925,43.295251
1314302,2014-541579,2014-02-27,1,Vente en l'état futur d'achèvement,5245808.15,51100,51454,Reims,51,51454000CK0366,2.0,Appartement,41.0,94.0,NaN,NaN,NaN,4.040248,49.253226


# pour faire un premier dans les lignes

### On ne garde pas les ligne où type local est nulle

In [19]:
immo2014 = immo2014[immo2014.type_local.notnull()]

### On dégage les  'Dépendances'

Il y a 1040 dépendances avec une surface réelle batie donnée contre 343017 ( 99.69 % de NA ).
On ne va pas prendre en comptes les dépendances. 

In [20]:
immo2014[immo2014.code_type_local == 3 ]["surface_reelle_bati"].isna().value_counts(dropna=False)

True     343017
False      1040
Name: surface_reelle_bati, dtype: int64

In [21]:
immo2014 = immo2014[immo2014.code_type_local != 3 ]

### On dégage les  'Local industriel. commercial ou assimilé'

In [22]:
immo2014[immo2014.code_type_local == 4 ]["surface_reelle_bati"].isna().value_counts(dropna=False)

False    68978
True     22260
Name: surface_reelle_bati, dtype: int64

In [23]:
immo2014 = immo2014[immo2014.code_type_local != 4 ]

Il y a 22260 'Local industriel. commercial ou assimilé' avec une surface réelle batie donnée contre 68978 ( 75 % de NA ). 
On ne va pas prendre en comptes les 'Local industriel. commercial ou assimilé'. 

### On dégage les  biens avec une valeur foncière pas donnée 

In [24]:
immo2014.valeur_fonciere.isna().value_counts()

False    906100
True       5745
Name: valeur_fonciere, dtype: int64

Il y a 0.63 de Na ( 5745 sur 906100 ) sur la valeur foncière pour les appartements et les maisons.
On enlève les lignes où la valeur foncière est nulle dans un premier temps

In [25]:
immo2014 = immo2014[immo2014.valeur_fonciere.notnull()]

### On dégage les biens avec une surface_reelle_bati = NA et un nombre de pices principales  = NA
cela concerne 64 ligne sur 906000 seulement

In [26]:
immo2014.isna().sum()

id_mutation                       0
date_mutation                     0
numero_disposition                0
nature_mutation                   0
valeur_fonciere                   0
code_postal                    1851
code_commune                      0
nom_commune                       0
code_departement                  0
id_parcelle                       0
code_type_local                   0
type_local                        0
surface_reelle_bati              65
nombre_pieces_principales        64
code_nature_culture          377667
nature_culture               377667
surface_terrain              377675
longitude                     36756
latitude                      36756
dtype: int64

In [27]:
immo2014 = immo2014[immo2014.surface_reelle_bati.notnull()]

In [28]:
immo2014.isna().sum()

id_mutation                       0
date_mutation                     0
numero_disposition                0
nature_mutation                   0
valeur_fonciere                   0
code_postal                    1851
code_commune                      0
nom_commune                       0
code_departement                  0
id_parcelle                       0
code_type_local                   0
type_local                        0
surface_reelle_bati               0
nombre_pieces_principales         0
code_nature_culture          377611
nature_culture               377611
surface_terrain              377619
longitude                     36751
latitude                      36751
dtype: int64

## On enlève les duplicates

####  avec une règle maison ( on prend le premier élement des groupes fait sur l'id de mutation )

In [29]:
nb_id_mutation = immo2014.groupby(['id_mutation']).size()
nonunique_id_mutation = nb_id_mutation[nb_id_mutation != 1]
nonunique_id_mutation = nonunique_id_mutation.index

In [30]:
immo2014[immo2014.id_mutation.isin(nonunique_id_mutation)].sort_values(by=['id_mutation'], ascending=False)

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,code_postal,code_commune,nom_commune,code_departement,id_parcelle,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,surface_terrain,longitude,latitude
2358982,2014-999991,2014-10-27,1,Vente,185000.00,93430,93079,Villetaneuse,93,930790000T0008,1.0,Maison,34.0,2.0,S,sols,155.0,2.338149,48.953243
2358981,2014-999991,2014-10-27,1,Vente,185000.00,93430,93079,Villetaneuse,93,930790000T0008,1.0,Maison,63.0,5.0,S,sols,155.0,2.338149,48.953243
2358965,2014-999980,2014-10-30,1,Vente,400000.00,93380,93059,Pierrefitte-sur-Seine,93,930590000G0255,2.0,Appartement,42.0,2.0,S,sols,500.0,2.373753,48.968294
2358963,2014-999980,2014-10-30,1,Vente,400000.00,93380,93059,Pierrefitte-sur-Seine,93,930590000G0255,2.0,Appartement,39.0,2.0,S,sols,500.0,2.373753,48.968294
2358955,2014-999980,2014-10-30,1,Vente,400000.00,93380,93059,Pierrefitte-sur-Seine,93,930590000G0255,2.0,Appartement,35.0,2.0,S,sols,500.0,2.373753,48.968294
2358956,2014-999980,2014-10-30,1,Vente,400000.00,93380,93059,Pierrefitte-sur-Seine,93,930590000G0255,2.0,Appartement,37.0,2.0,S,sols,500.0,2.373753,48.968294
2358957,2014-999980,2014-10-30,1,Vente,400000.00,93380,93059,Pierrefitte-sur-Seine,93,930590000G0255,2.0,Appartement,37.0,2.0,S,sols,500.0,2.373753,48.968294
2358964,2014-999980,2014-10-30,1,Vente,400000.00,93380,93059,Pierrefitte-sur-Seine,93,930590000G0255,2.0,Appartement,33.0,2.0,S,sols,500.0,2.373753,48.968294
248570,2014-99998,2014-05-16,1,Vente,160000.00,13005,13205,Marseille 5e Arrondissement,13,132058200E0429,2.0,Appartement,26.0,1.0,NaN,NaN,NaN,5.393567,43.294326
248569,2014-99998,2014-05-16,1,Vente,160000.00,13005,13205,Marseille 5e Arrondissement,13,132058200E0429,2.0,Appartement,25.0,1.0,NaN,NaN,NaN,5.393567,43.294326


In [31]:
immo2014 = immo2014.groupby('id_mutation').first().sort_values(by=['id_mutation'], ascending=False)

In [32]:
immo2014.shape

(711594, 18)

In [33]:
immo2014.isna().sum()

date_mutation                     0
numero_disposition                0
nature_mutation                   0
valeur_fonciere                   0
code_postal                    1511
code_commune                      0
nom_commune                       0
code_departement                  0
id_parcelle                       0
code_type_local                   0
type_local                        0
surface_reelle_bati               0
nombre_pieces_principales         0
code_nature_culture          339587
nature_culture               339587
surface_terrain              339588
longitude                     23558
latitude                      23558
dtype: int64

### On génère la variable mois et année ( début le janvier 2014 )

In [34]:
import datetime
immo2014['year'] = immo2014.date_mutation.map(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').year)
immo2014['month'] = immo2014.date_mutation.map(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').month)

In [35]:
immo2014.head()

,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,code_postal,code_commune,nom_commune,code_departement,id_parcelle,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,surface_terrain,longitude,latitude,year,month
id_mutation,,,,,,,,,,,,,,,,,,,,
2014-999999,2014-10-30,1,Vente en l'état futur d'achèvement,242000.0,93240,93072,Stains,93,930720000J0213,2.0,Appartement,81.0,4.0,NaN,NaN,NaN,2.380821,48.958099,2014,10
2014-999998,2014-11-19,1,Vente,116000.0,93300,93001,Aubervilliers,93,93001000AH0092,2.0,Appartement,47.0,2.0,NaN,NaN,NaN,2.392688,48.921551,2014,11
2014-999997,2014-11-05,1,Vente,29000.0,93300,93001,Aubervilliers,93,930010000J0059,2.0,Appartement,20.0,2.0,NaN,NaN,NaN,2.380943,48.917022,2014,11
2014-999996,2014-10-31,1,Vente,64000.0,93400,93070,Saint-Ouen-sur-Seine,93,930700000F0047,2.0,Appartement,22.0,2.0,NaN,NaN,NaN,NaN,NaN,2014,10
2014-999995,2014-11-13,1,Vente,110000.0,93300,93001,Aubervilliers,93,930010000K0177,2.0,Appartement,32.0,1.0,NaN,NaN,NaN,2.378502,48.912446,2014,11


In [36]:
immo2014.shape

(711594, 20)

### Nouveau tri dans les colonnes

In [37]:
immo2014_2 = immo2014

In [38]:
cols_to_keep = ["code_commune","code_postal","nom_commune","code_type_local", "type_local","nombre_pieces_principales","surface_reelle_bati","valeur_fonciere","month","year"]

In [39]:
immo2014_2 = immo2014_2[cols_to_keep]

### Mise en place d'un id de bien 

In [40]:
immo2014_2 = immo2014_2.reset_index()
immo2014_2.head()

,id_mutation,code_commune,code_postal,nom_commune,code_type_local,type_local,nombre_pieces_principales,surface_reelle_bati,valeur_fonciere,month,year
0,2014-999999,93072,93240,Stains,2.0,Appartement,4.0,81.0,242000.0,10,2014
1,2014-999998,93001,93300,Aubervilliers,2.0,Appartement,2.0,47.0,116000.0,11,2014
2,2014-999997,93001,93300,Aubervilliers,2.0,Appartement,2.0,20.0,29000.0,11,2014
3,2014-999996,93070,93400,Saint-Ouen-sur-Seine,2.0,Appartement,2.0,22.0,64000.0,10,2014
4,2014-999995,93001,93300,Aubervilliers,2.0,Appartement,1.0,32.0,110000.0,11,2014


In [41]:
immo2014_2['id_bien'] = immo2014_2["code_commune"] + "_" + immo2014_2["code_type_local"].astype(int).astype(str) + "_" + immo2014_2["nombre_pieces_principales"].astype(int).astype(str)
immo2014_2.head()

,id_mutation,code_commune,code_postal,nom_commune,code_type_local,type_local,nombre_pieces_principales,surface_reelle_bati,valeur_fonciere,month,year,id_bien
0,2014-999999,93072,93240,Stains,2.0,Appartement,4.0,81.0,242000.0,10,2014,93072_2_4
1,2014-999998,93001,93300,Aubervilliers,2.0,Appartement,2.0,47.0,116000.0,11,2014,93001_2_2
2,2014-999997,93001,93300,Aubervilliers,2.0,Appartement,2.0,20.0,29000.0,11,2014,93001_2_2
3,2014-999996,93070,93400,Saint-Ouen-sur-Seine,2.0,Appartement,2.0,22.0,64000.0,10,2014,93070_2_2
4,2014-999995,93001,93300,Aubervilliers,2.0,Appartement,1.0,32.0,110000.0,11,2014,93001_2_1


### Creation table bien 2014

In [42]:
cols_bien = ['code_commune','code_postal','nom_commune','nombre_pieces_principales','code_type_local','type_local','id_bien']

In [43]:
bien2014 = immo2014_2[cols_bien].drop_duplicates()

In [44]:
bien2014.head()

,code_commune,code_postal,nom_commune,nombre_pieces_principales,code_type_local,type_local,id_bien
0,93072,93240,Stains,4.0,2.0,Appartement,93072_2_4
1,93001,93300,Aubervilliers,2.0,2.0,Appartement,93001_2_2
3,93070,93400,Saint-Ouen-sur-Seine,2.0,2.0,Appartement,93070_2_2
4,93001,93300,Aubervilliers,1.0,2.0,Appartement,93001_2_1
5,93070,93400,Saint-Ouen-sur-Seine,4.0,1.0,Maison,93070_1_4


### On enlève les info redondantes dans la table immo2014

In [45]:
cols_bien.pop()

'id_bien'

In [46]:
immo2014_2 = immo2014_2.drop(columns = cols_bien).drop(columns = 'id_mutation')

In [47]:
bien2014.head()

,code_commune,code_postal,nom_commune,nombre_pieces_principales,code_type_local,type_local,id_bien
0,93072,93240,Stains,4.0,2.0,Appartement,93072_2_4
1,93001,93300,Aubervilliers,2.0,2.0,Appartement,93001_2_2
3,93070,93400,Saint-Ouen-sur-Seine,2.0,2.0,Appartement,93070_2_2
4,93001,93300,Aubervilliers,1.0,2.0,Appartement,93001_2_1
5,93070,93400,Saint-Ouen-sur-Seine,4.0,1.0,Maison,93070_1_4


### On génère un tableau bien_par_mois

In [48]:
id_bien = pd.DataFrame(data={'id_bien': immo2014_2['id_bien'].drop_duplicates()})

In [49]:
id_bien.head()

,id_bien
0,93072_2_4
1,93001_2_2
3,93070_2_2
4,93001_2_1
5,93070_1_4


In [50]:
id_bien.shape

(127506, 1)

In [51]:
mois = immo2014_2[['month','year']].drop_duplicates().sort_values(by="month")

In [52]:
id_bien["key"] = 0
mois["key"] = 0
bien_par_mois = id_bien.merge(mois, how='left', on = 'key').drop(columns = "key")

In [53]:
bien_par_mois.head()

,id_bien,month,year
0,93072_2_4,1,2014
1,93072_2_4,2,2014
2,93072_2_4,3,2014
3,93072_2_4,4,2014
4,93072_2_4,5,2014


In [54]:
bien_par_mois.shape

(1530072, 3)

### Group by (  "id_bien","month","year" ) du dataset immo

In [55]:
immoGroup2014 = immo2014_2

In [56]:
immoGroup2014.head()

,surface_reelle_bati,valeur_fonciere,month,year,id_bien
0,81.0,242000.0,10,2014,93072_2_4
1,47.0,116000.0,11,2014,93001_2_2
2,20.0,29000.0,11,2014,93001_2_2
3,22.0,64000.0,10,2014,93070_2_2
4,32.0,110000.0,11,2014,93001_2_1


In [57]:
immoGroup2014 = immoGroup2014.groupby(["id_bien","month","year"]).agg({'surface_reelle_bati':['mean','count'], 'valeur_fonciere':'mean'}).reset_index()

In [58]:
immoGroup2014.columns = ['id_bien','month','year','surface','nb_mutation','valeur']

In [59]:
immoGroup2014.head()

,id_bien,month,year,surface,nb_mutation,valeur
0,01001_1_3,2,2014,62.0,1,134750.000000
1,01001_1_4,1,2014,100.0,3,110666.666667
2,01001_1_5,10,2014,102.0,2,211500.000000
3,01001_1_6,7,2014,111.0,2,189910.000000
4,01001_1_6,9,2014,136.0,1,260000.000000


### Merge du dataset immo avec bien_par_mois

In [60]:
immoFinal = bien_par_mois.merge(immoGroup2014, how='left', on = ['id_bien','month','year'])
immoFinal.head()

,id_bien,month,year,surface,nb_mutation,valeur
0,93072_2_4,1,2014,70.000000,5.0,179202.804
1,93072_2_4,2,2014,72.666667,3.0,204337.850
2,93072_2_4,3,2014,74.500000,2.0,142500.000
3,93072_2_4,4,2014,79.000000,3.0,193708.880
4,93072_2_4,5,2014,68.750000,4.0,169725.465


In [61]:
immoFinal.fillna(0, inplace=True)

In [62]:
immoFinal.head()

,id_bien,month,year,surface,nb_mutation,valeur
0,93072_2_4,1,2014,70.000000,5.0,179202.804
1,93072_2_4,2,2014,72.666667,3.0,204337.850
2,93072_2_4,3,2014,74.500000,2.0,142500.000
3,93072_2_4,4,2014,79.000000,3.0,193708.880
4,93072_2_4,5,2014,68.750000,4.0,169725.465


In [63]:
immoFinal.to_csv('../../data/immobilier_2014_clean_month.csv',index=False)

In [ ]:
immoFinal